In [4]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import folium

# Load the trained models
models = {
    'Financial District': load_model('FinancialDistrict_gru_model.keras'),
    'Harlem': load_model('Harlem_gru_model.keras'),
    'Mid Town': load_model('MidTown_gru_model.keras'),
    'Upper East Side': load_model('UES_gru_model.keras'),
    'Upper West Side': load_model('UWS_gru_model.keras')
}

# Load data for each area directly from CSV files
data = {
    'Financial District': pd.read_csv('Financial District.csv').values,
    'Harlem': pd.read_csv('Harlem.csv').values,
    'Mid Town': pd.read_csv('Mid Town.csv').values,
    'Upper East Side': pd.read_csv('Upper East Side.csv').values,
    'Upper West Side': pd.read_csv('Upper West Side.csv').values
}


# Define the number of timesteps your model expects
timesteps = 4  # Adjust this to match your model's expected input
num_features = data['Financial District'].shape[1]  # Assuming all areas have the same number of features

# Reshape the data to match the model's expected input shape
reshaped_data = {}
for area, data_array in data.items():
    num_samples = data_array.shape[0] - timesteps + 1
    reshaped_data[area] = np.array([data_array[i:i+timesteps] for i in range(num_samples)])



# Make predictions
predictions = {}
for area, model in models.items():
    X = reshaped_data[area]  # Ensure X is the correct shape and preprocessed
    predictions[area] = model.predict(X)

# Example: Print predictions for each area
for area, prediction in predictions.items():
    print(f"{area}: {prediction[-1][0]:.2f} orders")


# Define coordinates for each area
coordinates = {
    'Financial District': (40.7074, -74.0113),
    'Harlem': (40.8116, -73.9465),
    'Mid Town': (40.7549, -73.9840),
    'Upper East Side': (40.7736, -73.9566),
    'Upper West Side': (40.7870, -73.9754)
}

# Create a Folium map centered around New York City
m = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

# Add markers for each area with demand predictions
for area, coords in coordinates.items():
    demand = predictions[area][-1][0]  # Get the latest prediction
    folium.CircleMarker(
        location=coords,
        radius=10,
        popup=f"{area}: {demand:.2f} orders",
        color='red',
        fill=True,
        fill_color='red'
    ).add_to(m)

# Save the map to an HTML file
m.save('demand_predictions_map.html')

# Display the map in a Jupyter Notebook (if applicable)
m

136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
136/136 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
135/135 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
132/132 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Financial District: -12.51 orders
Harlem: -5.40 orders
Mid Town: -14.16 orders
Upper East Side: -3.71 orders
Upper West Side: -5.45 orders
